In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import html
from html2text import HTML2Text
import time

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('unis-tech-cleaned.csv', index_col = 0)
df.head(3)

,Location,School,Course,Degree_type,Pace,Duration,Language,Mode,Description,Link,Country,City,Continent,Duration_years,InEnglish,Lang_primary
0,"Barcelona, Spain +1 More",Harbour.Space University,Bachelor of Computer Science,Bachelor,Full-time,3 years,English,Campus,Harbour.Space Computer Science Bachelor progra...,https://www.bachelorstudies.com/Bachelor-of-Co...,Spain,Barcelona,Europe,3.0,True,English
1,"Marbella, Spain",Schellhammer Business School,Bachelor in IT and Computing (Online),Bachelor,Full-time,3 years,English,Online,This IT and Computing Undergraduate degree off...,https://www.onlinestudies.com/Bachelor-in-IT-a...,Spain,Marbella,Europe,3.0,True,English
2,"Berlin, Germany",IU International University of Applied Science...,B.Sc. Software Development,BSc,Full-time,36 months,English,Online,Learn the skills needed to be a successful sof...,https://www.onlinestudies.com/B.Sc.-Software-D...,Germany,Berlin,Europe,3.0,True,English


In [6]:
df.shape

(1560, 16)

In [7]:
df.duplicated().sum()

0

In [8]:
df.replace("Unknown", np.nan, inplace = True)
df.isna().sum()

Location            0
School              0
Course              0
Degree_type         0
Pace              481
Duration          606
Language            0
Mode              102
Description         0
Link               12
Country             0
City                0
Continent           0
Duration_years      0
InEnglish           0
Lang_primary        0
dtype: int64

In [9]:
df.dropna(subset=["Link"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
#accept cookies - manual - not sure if it works without this
driver = webdriver.Chrome()
driver.get(df['Link'][0])

In [11]:
#keywords = ['Data','Business', 'Information', 'Analytics', 'Management']
keywords = ['Computer Science', 'Software', 'Development', 'Developer']
df1 = pd.DataFrame()
for i in range(0,len(df)):
    for kw in keywords:
        if kw in df['Course'][i]:
            df1 = pd.concat([df1, df.loc[i]], axis = 1)
            
df1 = df1.transpose()
df1.reset_index(drop = True, inplace = True)
print(df1.shape)
df1.head(3)

(496, 16)


,Location,School,Course,Degree_type,Pace,Duration,Language,Mode,Description,Link,Country,City,Continent,Duration_years,InEnglish,Lang_primary
0,"Barcelona, Spain +1 More",Harbour.Space University,Bachelor of Computer Science,Bachelor,Full-time,3 years,English,Campus,Harbour.Space Computer Science Bachelor progra...,https://www.bachelorstudies.com/Bachelor-of-Co...,Spain,Barcelona,Europe,3,True,English
1,"Berlin, Germany",IU International University of Applied Science...,B.Sc. Software Development,BSc,Full-time,36 months,English,Online,Learn the skills needed to be a successful sof...,https://www.onlinestudies.com/B.Sc.-Software-D...,Germany,Berlin,Europe,3,True,English
2,"Berlin, Germany",IU International University of Applied Science...,B.Sc. Software Development,BSc,Full-time,36 months,English,Online,Learn the skills needed to be a successful sof...,https://www.onlinestudies.com/B.Sc.-Software-D...,Germany,Berlin,Europe,3,True,English


In [12]:
df1.duplicated().sum()

27

In [13]:
df1.drop_duplicates(keep = 'first', inplace = True, ignore_index = True)

In [5]:
def get_intro(link):
    driver = webdriver.Chrome()
    driver.get(link)

    txt = []
    try:
        for i in html.unescape(driver.find_element(By.ID, 'introduction').get_attribute('innerHTML')).split('\n'):
            if len(i)>0:
                if i[0] != '<' and i[:3] != '   ':
                    txt.append(i)
        intro = ' '.join(txt)
    except:
        intro = 'Unknown'
            
    driver.quit()
    if len(intro)<20:
        intro = 'Unknown'
    
    return intro

In [20]:
intros = []
c = [0,0] #counter
for link in df1['Link']: #add [start:to] somehow?
    
    #scrape intro
    try:
        intro = get_intro(link)
    except:
        intro = 'Unknown' 
        
    intros.append(intro)
    
    #adjust counter
    c[0] += 1
    if c[0]%20 == 0:
        print(c)
    if intro != 'Unknown':
        c[1] += 1
        
    #end of loop
        
if len(intros) == len(df1):
    df1['Introduction'] = intros
df1.to_csv('developer-wintros-raw.csv')

[20, 17]
[40, 35]
[60, 51]
[80, 69]
[100, 88]
[120, 104]
[140, 123]
[160, 143]
[180, 163]
[200, 183]
[220, 203]
[240, 222]
[260, 242]
[280, 262]
[300, 282]
[320, 302]
[340, 322]
[360, 342]
[380, 362]
[400, 382]
[420, 402]
[440, 422]
[460, 442]


In [6]:
df1 = pd.read_csv('df1_final_dev.csv', index_col = 0)

In [36]:
df1.replace("Unknown", np.nan, inplace = True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469 entries, 0 to 468
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Location         469 non-null    object 
 1   School           469 non-null    object 
 2   Course           469 non-null    object 
 3   Degree_type      469 non-null    object 
 4   Pace             307 non-null    object 
 5   Duration         280 non-null    object 
 6   Language         469 non-null    object 
 7   Mode             443 non-null    object 
 8   Description      469 non-null    object 
 9   Link             469 non-null    object 
 10  Country          469 non-null    object 
 11  City             469 non-null    object 
 12  Continent        469 non-null    object 
 13  Duration_years   469 non-null    float64
 14  InEnglish        469 non-null    bool   
 15  Lang_primary     469 non-null    object 
 16  Introduction     469 non-null    object 
 17  Curriculum      

In [5]:
df1.dropna(subset = ['Introduction'], inplace = True)
df1.reset_index(drop = True, inplace = True)

In [8]:
def click_element(element_id,driver):
    element = driver.find_element(By.ID, element_id)
    driver.execute_script("arguments[0].scrollIntoView();", element)
    try:
        element.click()
        #print('a')
    except:
        driver.execute_script("window.scrollTo(0, window.scrollY + 100)")
        element = driver.find_element(By.ID, element_id)
        element.click()
        #print('b')
    else:
        pass
        #print(link)

In [9]:
def get_section_text(element_id,driver):
    h = HTML2Text()
    h.ignore_links = True
    text = ''
    for line in h.handle(driver.find_element(By.ID,element_id).get_attribute('innerHTML')):
            # handle() Function only accepts string as parameter            
        text += line   
    section = ''
    for i in text.split('\n'):
        section+=i.strip()        
        
    return section

In [21]:
def get_section(element_id, link):
    driver = webdriver.Chrome()
    driver.get(link)
    
    try:
        click_element(element_id,driver)
        time.sleep(3)
        section = get_section_text(element_id,driver)
    except:
        section = 'Unknown'
    
    if len(section)<50:
        time.sleep(10)
        try:
            section = get_section_text(element_id)
        except:
            section = 'Unknown'
         
    driver.quit()
    
    return section

In [22]:
def get_sections_list(link_list, element_id):
    sections = []
    c = [0,0]
    for link in link_list:
        try:
            section = get_section(element_id, link)
        except:
            section = 'Unknown'
        sections.append(section)

        c[0] += 1
        if c[0]%20 == 0:
            print(c)
        if section != "Unknown":
            c[1] += 1
        
    return sections

In [37]:
curriculum = get_sections_list(df['Link'], 'curriculum')

df['Curriculum'] = curriculum

[10, 9]
[20, 19]
[30, 29]
[40, 39]
[50, 49]
[60, 59]
[70, 69]
[80, 79]
[90, 89]
[100, 99]
[110, 109]
[120, 119]
[130, 129]
[140, 139]
[150, 149]
[160, 159]
[170, 169]
[180, 179]
[190, 189]
[200, 199]
[210, 209]
[220, 219]
[230, 229]
[240, 239]
[250, 249]
[260, 259]
[270, 269]
[280, 279]
[290, 289]
[300, 299]
[310, 309]
[320, 319]
[330, 329]
[340, 339]
[350, 349]
[360, 359]
[370, 369]
[380, 379]
[390, 389]
[400, 399]
[410, 409]
[420, 419]
[430, 429]
[440, 439]
[450, 449]
[460, 459]


In [28]:
careers = get_sections_list(df1['Link'], 'career_opportunities')

if len(careers) == len(df1):
    df1['Careers'] = careers

df1.to_csv('developer-wcar-raw.csv')

[20, 9]
[40, 18]
[60, 30]
[80, 42]
[100, 52]
[120, 64]
[140, 71]
[160, 85]
[180, 95]
[200, 97]
[220, 97]
[240, 98]
[260, 101]
[280, 101]
[300, 101]
[320, 101]
[340, 101]
[360, 101]
[380, 101]
[400, 101]
[420, 101]
[440, 101]
[460, 101]


In [27]:
careers

['Unknown',
 "### Career OpportunitiesWith a Bachelor's degree in Software Development, you can expect excellentcareer prospects across all industries, with lucrative opportunities foradvancement and the possibility to work anywhere on the globe. Some of themany career options waiting for you in the future are:**JavaScript Developer**Own and operate all JavaScript applications for a specific project or acrosscompany operations. Develop websites, design the architecture of web-orientedbackend and optimise apps and webpages in order to give users an easy andclear interface and browsing experience.**Frontend Engineer**Enjoy working with HTML, CSS and JavaScript? More interested in the softwareelements that users have direct interaction with? Then a career as a FrontendEngineer is the right choice for you. In this role, you’ll gain experience inthe development of graphic user interfaces, and work on having softwaresystems that run as smoothly as possible from a user’s perspective.**Backend

In [32]:
get_section('program_outcome',df1['Link'][1])

'Unknown'

In [35]:
outcome = get_sections_list(df1['Link'], 'program_outcome')

if len(outcome) == len(df1):
    df1['Program_outcome'] = outcome

[20, 3]
[40, 10]
[60, 14]
[80, 16]
[100, 24]
[120, 25]
[140, 28]
[160, 32]
[180, 36]
[200, 36]
[220, 36]
[240, 37]
[260, 40]
[280, 40]
[300, 40]
[320, 40]
[340, 40]
[360, 40]
[380, 40]
[400, 40]
[420, 40]
[440, 40]
[460, 40]


In [33]:
df1['Introduction'].isna().sum()

0

In [40]:
#fillna
c = 0
for i in range(0,len(df1)):
    if df1['Curriculum'].isna()[i]:
        try:
            df1['Curriculum'][i] = get_section('curriculum', df1['Link'][i])
        except:
            pass
       
    c += 1
    if c%50 == 0:
        print(c)

<ipython-input-40-e834a2d106db>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Curriculum'][i] = get_section('curriculum', df1['Link'][i])


50
100
150
200
250
300
350
400
450


In [37]:
df1.to_csv('df1_dev_final.csv')

In [ ]:
df1.replace("Unknown", np.nan, inplace = True)
df1.info()

In [55]:
df1_sub = df1.dropna(subset = ['Curriculum', 'Careers'])
df1_sub.reset_index(drop = True, inplace = True)

In [56]:
df1_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Location         115 non-null    object 
 1   School           115 non-null    object 
 2   Course           115 non-null    object 
 3   Degree_type      115 non-null    object 
 4   Pace             115 non-null    object 
 5   Duration         114 non-null    object 
 6   Language         115 non-null    object 
 7   Mode             115 non-null    object 
 8   Description      115 non-null    object 
 9   Link             115 non-null    object 
 10  Country          115 non-null    object 
 11  City             115 non-null    object 
 12  Continent        115 non-null    object 
 13  Duration_years   115 non-null    float64
 14  InEnglish        115 non-null    bool   
 15  Lang_primary     115 non-null    object 
 16  Introduction     115 non-null    object 
 17  Curriculum      

In [58]:
df1_sub.to_csv('df1_sub.csv')